In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import utils

In [ ]:
spotify = utils.auth()

In [ ]:
my_songs_sorted = utils.get_my_playlist_songs(spotify)

In [ ]:
my_songs_sorted[0]

In [ ]:
artists_with_songs = {}
for song in my_songs_sorted:
    for i,artist in enumerate(song['artists']):
        artist_id = artist['id']
        if artist_id in artists_with_songs:
            artists_with_songs[artist_id]['times'].append(song['time'])
            artists_with_songs[artist_id]['songs'].append(song['id'])
        else:
            artists_with_songs[artist_id] = {'times':[song['time']], 'songs': [song['id']], 'name':artist['name']}
my_artists = list(artists_with_songs.keys())

In [ ]:
artists_with_songs

In [ ]:
time_genres = []
for s in my_songs_sorted:
    for i,artist in enumerate(s['artists']):
        print(artist['name'])
        genres = spotify.artist(artist['id'])['genres']
        print(genres)
        for g in genres:
            time_genres.append({'time':s['time'], 'genre':g})

In [ ]:
import pandas as pd

current_year = int(time_genres[0]['time'][:4])
current_month = int(time_genres[0]['time'][5:7])

total_genre_counts = {}
monthly_genres = {current_year:{current_month:{}}}
for tg in time_genres:
    month = int(tg['time'][5:7])
    year = int(tg['time'][:4])
    g = tg['genre']
    if year != current_year and month != current_month:
        current_month = month
        current_year = year
    if (year in monthly_genres):
        if (month in monthly_genres[year]):
            if (g in monthly_genres[year][month]):
                monthly_genres[year][month][g] += 1
            else:
                monthly_genres[year][month][g] = 1
        else:
            monthly_genres[year][month] = {g:1}
    else:
        monthly_genres[year] = {month:{g:1}}

    if g in total_genre_counts:
        total_genre_counts[g] += 1
    else:
        total_genre_counts[g] = 1
sorted_total_genre_counts = sorted(total_genre_counts.items(), key = lambda g : total_genre_counts[g[0]])

monthly_genre_df = pd.DataFrame({'year':[], 'month':[], 'genre':[], 'count':[]})
for y in monthly_genres:
    for m in monthly_genres[y]:
        for g,c in monthly_genres[y][m].items():
            monthly_genre_df.loc[len(monthly_genre_df)] = [y, m, g, c]

In [ ]:
def month_year_iter(start_month, start_year, end_month, end_year):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range(ym_start, ym_end):
        y, m = divmod(ym, 12)
        yield y, m+1

first_year = int(time_genres[0]['time'][:4])
last_year = int(time_genres[-1]['time'][:4])
first_month = int(time_genres[0]['time'][5:7])
last_month = int(time_genres[-1]['time'][5:7])

genre_count_lists = {}
elapsed_months = 0
yearmonths = []
n_genre_keep_per_month = 2
top_genres = []
for y in range(first_year, last_year+1):
    year_data = monthly_genre_df[monthly_genre_df['year'] == y]
    for m in range(1,13):
        if not ((y == first_year and m < first_month) or (y == last_year and m > last_month)):
            month_data = year_data[year_data['month'] == m]
            top_n = month_data.nlargest(n_genre_keep_per_month, 'count')['genre']
            for g in top_n:
                if g not in top_genres:
                    top_genres.append(g)

            for row in month_data.iterrows():
                g = row[1]['genre']
                c = row[1]['count']
                if g in genre_count_lists:
                    genre_count_lists[g].append(c)
                else:
                    genre_count_lists[g] = [0]*elapsed_months + [c]
            elapsed_months += 1
            yearmonths.append((y, m))
for g,counts in genre_count_lists.items():
    counts += [0] * (elapsed_months - len(counts))

In [ ]:
sorted_total_genre_counts

In [ ]:
len(sorted_total_genre_counts)

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(18,6))

n = 10
top_n_genres = [s[0] for s in sorted_total_genre_counts[-n:]]
yearmonth_strings = [str(y)+'-'+str(m) for y,m in yearmonths]
for g,counts in genre_count_lists.items():
    if g in top_n_genres:
        plt.plot(yearmonth_strings, counts, label=g)
plt.legend()
plt.xticks(rotation=60, ha='right')
plt.show()